In [1]:
from datasets import load_dataset
import tensorflow as tf
from transformers import MBart50Tokenizer, TFTrainingArguments, TFMBartForConditionalGeneration, DataCollatorForSeq2Seq, create_optimizer
import pandas as pd

c:\Users\tudor\anaconda3\envs\y2c\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# Load dataset
eng_nl = load_dataset("kde4", lang1="en", lang2="nl", download_mode="force_redownload")

# Checking the quality of translation
for i in range(100):
    print(f"Example {i}:")
    print(eng_nl["train"][i]["translation"])
    print()

kde4.py:   0%|          | 0.00/4.25k [00:00<?, ?B/s]

README.md:   0%|          | 0.00/5.10k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/192060 [00:00<?, ? examples/s]

Example 0:
{'en': 'Lauri Watts', 'nl': 'Lauri Watts'}

Example 1:
{'en': '& Lauri. Watts. mail;', 'nl': '& Lauri.Watts.mail;'}

Example 2:
{'en': 'ROLES_OF_TRANSLATORS', 'nl': '& Niels.Reedijk; Bram.Schoenmakers; Natalie.Koning;'}

Example 3:
{'en': '2006-02-26 3.5.1', 'nl': '2006-02-26 3.5.1'}

Example 4:
{'en': 'The Babel & konqueror; plugin gives you quick access to the Babelfish translation service.', 'nl': 'De plugin Babel voor & konqueror; geeft u snel en gemakkelijk toegang tot de Babelfish-vertaaldienst.'}

Example 5:
{'en': 'KDE', 'nl': 'KDE'}

Example 6:
{'en': 'kdeaddons', 'nl': 'kdeaddons'}

Example 7:
{'en': 'konqueror', 'nl': 'konqueror'}

Example 8:
{'en': 'plugins', 'nl': 'plugins'}

Example 9:
{'en': 'babelfish', 'nl': 'babelfish'}

Example 10:
{'en': 'translate', 'nl': 'translate'}

Example 11:
{'en': 'The Babel & konqueror; plugin', 'nl': 'De & konqueror; Babel-plugin'}

Example 12:
{'en': 'Using the Babelfish plugin', 'nl': 'De Babelfish-plugin gebruiken'}

Example 

In [3]:
# 1. Split the dataset for fine tuning
split_datasets = eng_nl["train"].train_test_split(test_size=0.1) # 90% train, 10% test

# 2. Initialize TensorFlow model
model = TFMBartForConditionalGeneration.from_pretrained("facebook/mbart-large-cc25")
tokenizer = MBart50Tokenizer.from_pretrained(
    "facebook/mbart-large-50",
    src_lang="en_XX",
    tgt_lang="nl_XX"
)

# 3. Preprocessing function
def preprocess(examples):
    inputs = [ex["en"] for ex in examples["translation"]]
    targets = [ex["nl"] for ex in examples["translation"]]
    
    model_inputs = tokenizer(
        inputs,
        max_length=128,
        truncation=True,
        padding="max_length"
    )
    
    with tokenizer.as_target_tokenizer():
        labels = tokenizer(
            targets,
            max_length=128,
            truncation=True,
            padding="max_length"
        )
    
    return {
        "input_ids": model_inputs["input_ids"],
        "attention_mask": model_inputs["attention_mask"],
        "labels": labels["input_ids"]
    }

# 4. Prepare TF Dataset
data_collator = DataCollatorForSeq2Seq(tokenizer, model=model, return_tensors="tf")

tokenized_datasets = split_datasets.map(
    preprocess,
    batched=True,
    remove_columns=split_datasets["train"].column_names
)

tf_train_dataset = model.prepare_tf_dataset(
    tokenized_datasets["train"],
    collate_fn=data_collator,
    batch_size=8,
    shuffle=True
)

tf_val_dataset = model.prepare_tf_dataset(
    tokenized_datasets["test"],
    collate_fn=data_collator,
    batch_size=8
)

# 5. Configure Training
num_train_steps = len(tf_train_dataset) * 10

optimizer, lr_schedule = create_optimizer(
    init_lr=3e-5,
    num_train_steps=num_train_steps,
    weight_decay_rate=0.05,
    num_warmup_steps=0,  # 10% warmup
)

loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
model.compile(optimizer=optimizer, loss=loss, metrics=['accuracy'])

# 6. Train the model
model.fit(
    tf_train_dataset,
    validation_data=tf_val_dataset,
    epochs=13,
    steps_per_epoch=200,
    validation_steps=200
)

# Save the tokenizer
tokenizer.save_pretrained("model_en_nl")

# Save the model
model.save_pretrained("model_en_nl")

2025-04-03 08:10:21.950971: W tensorflow/core/common_runtime/gpu/gpu_bfc_allocator.cc:47] Overriding orig_value setting because the TF_FORCE_GPU_ALLOW_GROWTH environment variable is set. Original config value was 0.
I0000 00:00:1743667821.951286 1150602 gpu_device.cc:2019] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 46672 MB memory:  -> device: 0, name: NVIDIA RTX 6000 Ada Generation, pci bus id: 0000:01:00.0, compute capability: 8.9
All model checkpoint layers were used when initializing TFMBartForConditionalGeneration.

All the layers of TFMBartForConditionalGeneration were initialized from the model checkpoint at facebook/mbart-large-cc25.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFMBartForConditionalGeneration for predictions without further training.


Map:   0%|          | 0/172854 [00:00<?, ? examples/s]

/usr/local/lib/python3.11/dist-packages/transformers/tokenization_utils_base.py:3970: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(


Map:   0%|          | 0/19206 [00:00<?, ? examples/s]

Epoch 1/13


/usr/local/lib/python3.11/dist-packages/tensorflow/python/framework/indexed_slices.py:446: UserWarning: Converting sparse IndexedSlices to a dense Tensor with 256027648 elements. This may consume a large amount of memory.
  warnings.warn(


200/200 [==============================] - 99s 380ms/step - loss: 4.3371 - accuracy: 0.7007 - val_loss: 1.0379 - val_accuracy: 0.9358
Epoch 2/13
200/200 [==============================] - 72s 363ms/step - loss: 0.8228 - accuracy: 0.9316 - val_loss: 0.4424 - val_accuracy: 0.9433
Epoch 3/13
200/200 [==============================] - 73s 365ms/step - loss: 0.4663 - accuracy: 0.9432 - val_loss: 0.2961 - val_accuracy: 0.9538
Epoch 4/13
200/200 [==============================] - 73s 364ms/step - loss: 0.3381 - accuracy: 0.9483 - val_loss: 0.2679 - val_accuracy: 0.9570
Epoch 5/13
200/200 [==============================] - 73s 366ms/step - loss: 0.3102 - accuracy: 0.9519 - val_loss: 0.2449 - val_accuracy: 0.9610
Epoch 6/13
200/200 [==============================] - 73s 366ms/step - loss: 0.3087 - accuracy: 0.9528 - val_loss: 0.2483 - val_accuracy: 0.9583
Epoch 7/13
200/200 [==============================] - 73s 364ms/step - loss: 0.2912 - accuracy: 0.9544 - val_loss: 0.2263 - val_accuracy: 0.9

/usr/local/lib/python3.11/dist-packages/transformers/configuration_utils.py:397: UserWarning: Some non-default generation parameters are set in the model config. These should go into either a) `model.generation_config` (as opposed to `model.config`); OR b) a GenerationConfig file (https://huggingface.co/docs/transformers/generation_strategies#save-a-custom-decoding-strategy-with-your-model).This warning will become an exception in the future.
Non-default generation parameters: {'max_length': 1024, 'num_beams': 5}
  warnings.warn(


In [4]:
# 1. Split the dataset for fine tuning
split_datasets = eng_nl["train"].train_test_split(test_size=0.1) # 90% train, 10% test

# 2. Initialize TensorFlow model
model_nl_en = TFMBartForConditionalGeneration.from_pretrained("facebook/mbart-large-cc25")
tokenizer_nl_en = MBart50Tokenizer.from_pretrained(
    "facebook/mbart-large-50",
    src_lang="nl_XX",  # Dutch is now the source language
    tgt_lang="en_XX"   # English is now the target language
)

# 3. Preprocessing function
def preprocess_nl_en(examples):
    # Swap source and target languages
    inputs = [ex["nl"] for ex in examples["translation"]]  # Now using Dutch as input
    targets = [ex["en"] for ex in examples["translation"]]  # Now using English as target
    
    model_inputs = tokenizer_nl_en(
        inputs,
        max_length=128,
        truncation=True,
        padding="max_length"
    )
    
    with tokenizer_nl_en.as_target_tokenizer():
        labels = tokenizer_nl_en(
            targets,
            max_length=128,
            truncation=True,
            padding="max_length"
        )
    
    return {
        "input_ids": model_inputs["input_ids"],
        "attention_mask": model_inputs["attention_mask"],
        "labels": labels["input_ids"]
    }

# 4. Prepare TF Dataset
data_collator_nl_en = DataCollatorForSeq2Seq(tokenizer_nl_en, model=model_nl_en, return_tensors="tf")
tokenized_datasets_nl_en = split_datasets.map(
    preprocess_nl_en,
    batched=True,
    remove_columns=split_datasets["train"].column_names
)

tf_train_dataset_nl_en = model_nl_en.prepare_tf_dataset(
    tokenized_datasets_nl_en["train"],
    collate_fn=data_collator_nl_en,
    batch_size=8,
    shuffle=True
)

tf_val_dataset_nl_en = model_nl_en.prepare_tf_dataset(
    tokenized_datasets_nl_en["test"],
    collate_fn=data_collator_nl_en,
    batch_size=8
)

# 5. Configure Training
num_train_steps = len(tf_train_dataset_nl_en) * 10
optimizer, lr_schedule = create_optimizer(
    init_lr=3e-5,
    num_train_steps=num_train_steps,
    weight_decay_rate=0.05,
    num_warmup_steps=int(num_train_steps * 0.1),  # 10% warmup
)

loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
model_nl_en.compile(optimizer=optimizer, loss=loss, metrics=['accuracy'])

# 6. Train the model
model_nl_en.fit(
    tf_train_dataset_nl_en,
    validation_data=tf_val_dataset_nl_en,
    epochs=13,
    steps_per_epoch=200,
    validation_steps=200
)

# Save the tokenizer
tokenizer.save_pretrained("model_nl_en")

# Save the model
model.save_pretrained("model_nl_en")

All model checkpoint layers were used when initializing TFMBartForConditionalGeneration.

All the layers of TFMBartForConditionalGeneration were initialized from the model checkpoint at facebook/mbart-large-cc25.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFMBartForConditionalGeneration for predictions without further training.


Map:   0%|          | 0/172854 [00:00<?, ? examples/s]

/usr/local/lib/python3.11/dist-packages/transformers/tokenization_utils_base.py:3970: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(


Map:   0%|          | 0/19206 [00:00<?, ? examples/s]

Epoch 1/13


/usr/local/lib/python3.11/dist-packages/tensorflow/python/framework/indexed_slices.py:446: UserWarning: Converting sparse IndexedSlices to a dense Tensor with 256027648 elements. This may consume a large amount of memory.
  warnings.warn(


200/200 [==============================] - 94s 381ms/step - loss: 40.2269 - accuracy: 0.0126 - val_loss: 17.8227 - val_accuracy: 0.0209
Epoch 2/13
200/200 [==============================] - 72s 362ms/step - loss: 18.0805 - accuracy: 0.0137 - val_loss: 14.0046 - val_accuracy: 0.0207
Epoch 3/13
200/200 [==============================] - 72s 363ms/step - loss: 14.0246 - accuracy: 0.0143 - val_loss: 11.9279 - val_accuracy: 0.0208
Epoch 4/13
200/200 [==============================] - 73s 364ms/step - loss: 12.0697 - accuracy: 0.0126 - val_loss: 10.5704 - val_accuracy: 0.0261
Epoch 5/13
200/200 [==============================] - 73s 363ms/step - loss: 10.5271 - accuracy: 0.0146 - val_loss: 9.0743 - val_accuracy: 0.0243
Epoch 6/13
200/200 [==============================] - 72s 362ms/step - loss: 8.9076 - accuracy: 0.0738 - val_loss: 7.2989 - val_accuracy: 0.8829
Epoch 7/13
200/200 [==============================] - 72s 361ms/step - loss: 7.0614 - accuracy: 0.6702 - val_loss: 5.3268 - val_accu

/usr/local/lib/python3.11/dist-packages/transformers/configuration_utils.py:397: UserWarning: Some non-default generation parameters are set in the model config. These should go into either a) `model.generation_config` (as opposed to `model.config`); OR b) a GenerationConfig file (https://huggingface.co/docs/transformers/generation_strategies#save-a-custom-decoding-strategy-with-your-model).This warning will become an exception in the future.
Non-default generation parameters: {'max_length': 1024, 'num_beams': 5}
  warnings.warn(
